In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from pathlib import Path
# sys.path.insert(0, "/Users/poorna/Downloads/CE-updated/calibrated_explanations/src")

# Import the necessary libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from calibrated_explanations import CalibratedExplainer, __version__

print(f"calibrated_explanations {__version__}")

calibrated_explanations v0.10.1


In [3]:
# Load and preprocess the data
num_to_test = 10  # number of instances to test, one from each class
dataset = "diabetes_full"
delimiter = ","
model = "RF"

filename = "../data/diabetes_full.csv"
df = pd.read_csv(filename, delimiter=delimiter)
target = "Y"
X, y = df.drop(target, axis=1), df[target]
no_of_classes = len(np.unique(y))
no_of_features = X.shape[1]
no_of_instances = X.shape[0]

# find categorical features
categorical_features = [i for i in range(no_of_features) if len(np.unique(X.iloc[:, i])) < 10]

# select test instances from each class and split into train, cal and test
idx = np.argsort(y.values).astype(int)
X, y = X.values[idx, :], y.values[idx]
test_index = np.array(
    [
        *range(int(num_to_test / 2)),
        *range(no_of_instances - 1, no_of_instances - int(num_to_test / 2) - 1, -1),
    ]
)
train_index = np.setdiff1d(np.array(range(no_of_instances)), test_index)
X_train, X_test = X[train_index, :], X[test_index, :]
y_train, y_test = y[train_index], y[test_index]
X_prop_train, X_cal, y_prop_train, y_cal = train_test_split(
    X_train, y_train, test_size=0.33, random_state=42, stratify=y_train
)

In [4]:
# Train the model and create the explainer
model = RandomForestClassifier()

model.fit(X_prop_train, y_prop_train)

ce = CalibratedExplainer(
    model,
    X_cal,
    y_cal,
    feature_names=df.columns,
    categorical_features=categorical_features,
    class_labels={0: "Non-diabetic", 1: "Diabetic"},
)

c:\Users\loftuw\AppData\Local\anaconda3\envs\calibrated_explanations_env\lib\importlib\__init__.py:126: UserWarning: Cache backend fallback: using minimal in-package LRU/TTL implementation due to missing 'cachetools'
  return _bootstrap._gcd_import(name[level:], package, level)


In [5]:
factual_explanations = ce.explain_factual(X_test)
print("Probability [lower and upper bound] for Diabetic:")
print(
    *zip(
        [
            f"Instance {i}: {exp.prediction['predict']:.3f} [{exp.prediction['low']:5.3f}, {exp.prediction['high']:5.3f}]"
            for i, exp in enumerate(factual_explanations)
        ]
    ),
    sep="\n",
)

Probability [lower and upper bound] for Diabetic:
('Instance 0: 0.077 [0.040, 0.080]',)
('Instance 1: 0.054 [0.033, 0.056]',)
('Instance 2: 0.065 [0.033, 0.067]',)
('Instance 3: 0.404 [0.375, 0.423]',)
('Instance 4: 0.778 [0.773, 0.795]',)
('Instance 5: 0.778 [0.773, 0.795]',)
('Instance 6: 0.778 [0.773, 0.795]',)
('Instance 7: 0.778 [0.773, 0.795]',)
('Instance 8: 0.397 [0.364, 0.419]',)
('Instance 9: 0.778 [0.773, 0.795]',)


Example demonstrating the new to_narrative API.

This shows the clean API for generating narratives from calibrated explanations.

In [6]:
template_path = "src/calibrated_explanations/templates/narrative_template.yaml"

# The new to_narrative method provides a clean API:
explanations = factual_explanations

# Basic usage
narratives = explanations[0].to_narrative(
    template_path=template_path,
    expertise_level=("beginner", "advanced"),
    output_format="text",
)
print(narratives)


Instance 0

Factual Explanation (Advanced):
--------------------------------------------------------------------------------
Prediction: Diabetic
Calibrated Probability: 0.077
Prediction Interval: [0.040, 0.080]

Factors increasing Calibrated Probability:
DiabetesPedigreeFunction (1.27) > 0.72 — weight ≈ 0.027 [0.025, 0.066],

Factors decreasing Calibrated Probability:
BMI (25.1) <= 30.25 — weight ≈ -0.325 [-0.344, -0.297],
Pregnancies (1.0) <= 6.50 — weight ≈ -0.192 [-0.204, -0.159],
Glucose (90.0) <= 102.50 — weight ≈ -0.182 [-0.196, -0.135],
Insulin (59.0) <= 128.00 — weight ≈ -0.134 [-0.156, -0.061],
SkinThickness (18.0) <= 28.50 — weight ≈ -0.071 [-0.089, 0.015], [⚠️ direction uncertain]
Age (25.0) <= 27.50 — weight ≈ -0.061 [-0.066, -0.030],

Factual Explanation (Beginner):
--------------------------------------------------------------------------------
Prediction: Diabetic
Calibrated Probability: 0.077 [0.040, 0.080]

Factors increasing Calibrated Probability:
* DiabetesPedigre

C:\Users\loftuw\Documents\Github\calibrated_explanations\src\calibrated_explanations\explanations\explanation.py:508: UserWarning: Narrative template fallback: default template used because provided relative path was missing
  result = plugin.plot(


In [7]:
# Different output formats:

# 1. DataFrame (default) - returns pandas DataFrame
df_narratives = explanations.to_narrative(
    template_path=template_path,
    expertise_level=("beginner", "advanced"),
    output_format="dataframe"
)
df_narratives

C:\Users\loftuw\Documents\Github\calibrated_explanations\src\calibrated_explanations\explanations\explanations.py:1098: UserWarning: Narrative template fallback: default template used because provided relative path was missing
  return plugin.plot(


,instance_index,factual_explanation_beginner,factual_explanation_advanced,expertise_level,problem_type
0,0,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
1,1,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
2,2,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
3,3,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
4,4,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
5,5,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
6,6,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
7,7,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
8,8,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
9,9,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification


In [8]:
# 2. Text - returns formatted text string
text_narratives = explanations.to_narrative(
    template_path=template_path,
    expertise_level="beginner",
    output_format="text"
)
print(text_narratives)


Instance 0

Factual Explanation (Beginner):
--------------------------------------------------------------------------------
Prediction: Diabetic
Calibrated Probability: 0.077 [0.040, 0.080]

Factors increasing Calibrated Probability:
* DiabetesPedigreeFunction (1.27) > 0.72,

Factors decreasing Calibrated Probability:
* BMI (25.1) <= 30.25,
* Pregnancies (1.0) <= 6.50,
* Glucose (90.0) <= 102.50,
* Insulin (59.0) <= 128.00,
* SkinThickness (18.0) <= 28.50, [⚠️ direction uncertain]
* Age (25.0) <= 27.50,

Instance 1

Factual Explanation (Beginner):
--------------------------------------------------------------------------------
Prediction: Diabetic
Calibrated Probability: 0.054 [0.033, 0.056]

Factors increasing Calibrated Probability:
* Pregnancies (0.0) <= 6.50,
* Age (21.0) <= 27.50,
* BMI (21.8) <= 30.25,
* DiabetesPedigreeFunction (0.45) <= 0.72, [⚠️ direction uncertain]
* Insulin (105.0) <= 128.00, [⚠️ direction uncertain]
* SkinThickness (13.0) <= 28.50, [⚠️ direction uncertain

C:\Users\loftuw\Documents\Github\calibrated_explanations\src\calibrated_explanations\explanations\explanations.py:1098: UserWarning: Narrative template fallback: default template used because provided relative path was missing
  return plugin.plot(


In [9]:
# 3. HTML - returns HTML table
html_narratives = explanations.to_narrative(
    template_path=template_path,
    expertise_level=("beginner", "intermediate", "advanced"),
    output_format="html"
)
from IPython.display import HTML
HTML(html_narratives)

C:\Users\loftuw\Documents\Github\calibrated_explanations\src\calibrated_explanations\explanations\explanations.py:1098: UserWarning: Narrative template fallback: default template used because provided relative path was missing
  return plugin.plot(


instance_index,factual_explanation_beginner,factual_explanation_intermediate,factual_explanation_advanced,expertise_level,problem_type
0,"Prediction: DiabeticCalibrated Probability: 0.077 [0.040, 0.080]Factors increasing Calibrated Probability:* DiabetesPedigreeFunction (1.27) > 0.72,Factors decreasing Calibrated Probability:* BMI (25.1) <= 30.25,* Pregnancies (1.0) <= 6.50,* Glucose (90.0) <= 102.50,* Insulin (59.0) <= 128.00,* SkinThickness (18.0) <= 28.50, [⚠️ direction uncertain]* Age (25.0) <= 27.50,","Prediction: DiabeticCalibrated Probability: 0.077 [0.040, 0.080]Factors increasing Calibrated Probability:* DiabetesPedigreeFunction (1.27) > 0.72 — weight ≈ 0.027,Factors decreasing Calibrated Probability:* BMI (25.1) <= 30.25 — weight ≈ -0.325,* Pregnancies (1.0) <= 6.50 — weight ≈ -0.192,* Glucose (90.0) <= 102.50 — weight ≈ -0.182,* Insulin (59.0) <= 128.00 — weight ≈ -0.134,* SkinThickness (18.0) <= 28.50 — weight ≈ -0.071, [⚠️ direction uncertain]* Age (25.0) <= 27.50 — weight ≈ -0.061,","Prediction: DiabeticCalibrated Probability: 0.077Prediction Interval: [0.040, 0.080]Factors increasing Calibrated Probability:DiabetesPedigreeFunction (1.27) > 0.72 — weight ≈ 0.027 [0.025, 0.066],Factors decreasing Calibrated Probability:BMI (25.1) <= 30.25 — weight ≈ -0.325 [-0.344, -0.297],Pregnancies (1.0) <= 6.50 — weight ≈ -0.192 [-0.204, -0.159],Glucose (90.0) <= 102.50 — weight ≈ -0.182 [-0.196, -0.135],Insulin (59.0) <= 128.00 — weight ≈ -0.134 [-0.156, -0.061],SkinThickness (18.0) <= 28.50 — weight ≈ -0.071 [-0.089, 0.015], [⚠️ direction uncertain]Age (25.0) <= 27.50 — weight ≈ -0.061 [-0.066, -0.030],","beginner, intermediate, advanced",binary_classification
1,"Prediction: DiabeticCalibrated Probability: 0.054 [0.033, 0.056]Factors increasing Calibrated Probability:* Pregnancies (0.0) <= 6.50,* Age (21.0) <= 27.50,* BMI (21.8) <= 30.25,* DiabetesPedigreeFunction (0.45) <= 0.72, [⚠️ direction uncertain]* Insulin (105.0) <= 128.00, [⚠️ direction uncertain]* SkinThickness (13.0) <= 28.50, [⚠️ direction uncertain]* BloodPressure (56.0) <= 69.00, [⚠️ direction uncertain]Factors decreasing Calibrated Probability:","Prediction: DiabeticCalibrated Probability: 0.054 [0.033, 0.056]Factors increasing Calibrated Probability:* Pregnancies (0.0) <= 6.50 — weight ≈ -0.220,* Age (21.0) <= 27.50 — weight ≈ -0.084,* BMI (21.8) <= 30.25 — weight ≈ -0.072,* DiabetesPedigreeFunction (0.45) <= 0.72 — weight ≈ -0.032, [⚠️ direction uncertain]* Insulin (105.0) <= 128.00 — weight ≈ -0.029, [⚠️ direction uncertain]* SkinThickness (13.0) <= 28.50 — weight ≈ -0.023, [⚠️ direction uncertain]* BloodPressure (56.0) <= 69.00 — weight ≈ -0.022, [⚠️ direction uncertain]Factors decreasing Calibrated Probability:","Prediction: DiabeticCalibrated Probability: 0.054Prediction Interval: [0.033, 0.056]Factors increasing Calibrated Probability:Pregnancies (0.0) <= 6.50 — weight ≈ -0.220 [-0.228, -0.200],Age (21.0) <= 27.50 — weight ≈ -0.084 [-0.089, -0.053],BMI (21.8) <= 30.25 — weight ≈ -0.072 [-0.078, -0.030],DiabetesPedigreeFunction (0.45) <= 0.72 — weight ≈ -0.032 [-0.036, 0.014], [⚠️ direction uncertain]Insulin (105.0) <= 128.00 — weight ≈ -0.029 [-0.033, 0.014], [⚠️ direction uncertain]SkinThickness (13.0) <= 28.50 — weight ≈ -0.023 [-0.027, 0.019], [⚠️ direction uncertain]BloodPressure (56.0) <= 69.00 — weight ≈ -0.022 [-0.025, 0.017], [⚠️ direction uncertain]Factors decreasing Calibrated Probability:","beginner, intermediate, advanced",binary_classification
2,"Prediction: DiabeticCalibrated Probability: 0.065 [0.033, 0.067]Factors increasing Calibrated Probability:* Pregnancies (0.0) <= 6.50,* Glucose (74.0) <= 102.50,* BMI (27.8) <= 30.25,* Age (22.0) <= 27.50,* BloodPressure (52.0) <= 69.00,* Insulin (36.0) <= 128.00, [⚠️ direction uncertain]* SkinThickness (10.0) <= 28.50,* DiabetesPedigreeFunction (0.27) <= 0.72, [⚠️ direction uncertain]Factors decreasing Calibrated Probability:","Prediction: DiabeticCalibrated Probability: 0.065 [0.033, 0.067]Factor

In [10]:
# 4. Dictionary - returns list of dictionaries
dict_narratives = explanations.to_narrative(
    template_path=template_path,
    expertise_level="advanced",
    output_format="dict"
)
dict_narratives

C:\Users\loftuw\Documents\Github\calibrated_explanations\src\calibrated_explanations\explanations\explanations.py:1098: UserWarning: Narrative template fallback: default template used because provided relative path was missing
  return plugin.plot(


[{'instance_index': 0,
  'factual_explanation_advanced': 'Prediction: Diabetic\nCalibrated Probability: 0.077\nPrediction Interval: [0.040, 0.080]\n\nFactors increasing Calibrated Probability:\nDiabetesPedigreeFunction (1.27) > 0.72 — weight ≈ 0.027 [0.025, 0.066],\n\nFactors decreasing Calibrated Probability:\nBMI (25.1) <= 30.25 — weight ≈ -0.325 [-0.344, -0.297],\nPregnancies (1.0) <= 6.50 — weight ≈ -0.192 [-0.204, -0.159],\nGlucose (90.0) <= 102.50 — weight ≈ -0.182 [-0.196, -0.135],\nInsulin (59.0) <= 128.00 — weight ≈ -0.134 [-0.156, -0.061],\nSkinThickness (18.0) <= 28.50 — weight ≈ -0.071 [-0.089, 0.015], [⚠️ direction uncertain]\nAge (25.0) <= 27.50 — weight ≈ -0.061 [-0.066, -0.030],',
  'expertise_level': ('advanced',),
  'problem_type': 'binary_classification'},
 {'instance_index': 1,
  'factual_explanation_advanced': 'Prediction: Diabetic\nCalibrated Probability: 0.054\nPrediction Interval: [0.033, 0.056]\n\nFactors increasing Calibrated Probability:\nPregnancies (0.0) <=

In [11]:
# Different expertise levels:

# Single level
beginner_only = explanations.to_narrative(
    expertise_level="beginner",
    output_format="text",
    template_path=template_path
)
print(beginner_only)


Instance 0

Factual Explanation (Beginner):
--------------------------------------------------------------------------------
Prediction: Diabetic
Calibrated Probability: 0.077 [0.040, 0.080]

Factors increasing Calibrated Probability:
* DiabetesPedigreeFunction (1.27) > 0.72,

Factors decreasing Calibrated Probability:
* BMI (25.1) <= 30.25,
* Pregnancies (1.0) <= 6.50,
* Glucose (90.0) <= 102.50,
* Insulin (59.0) <= 128.00,
* SkinThickness (18.0) <= 28.50, [⚠️ direction uncertain]
* Age (25.0) <= 27.50,

Instance 1

Factual Explanation (Beginner):
--------------------------------------------------------------------------------
Prediction: Diabetic
Calibrated Probability: 0.054 [0.033, 0.056]

Factors increasing Calibrated Probability:
* Pregnancies (0.0) <= 6.50,
* Age (21.0) <= 27.50,
* BMI (21.8) <= 30.25,
* DiabetesPedigreeFunction (0.45) <= 0.72, [⚠️ direction uncertain]
* Insulin (105.0) <= 128.00, [⚠️ direction uncertain]
* SkinThickness (13.0) <= 28.50, [⚠️ direction uncertain

C:\Users\loftuw\Documents\Github\calibrated_explanations\src\calibrated_explanations\explanations\explanations.py:1098: UserWarning: Narrative template fallback: default template used because provided relative path was missing
  return plugin.plot(


In [12]:
# Multiple levels
all_levels = explanations.to_narrative(
    expertise_level=("beginner", "intermediate", "advanced"),
    output_format="dataframe",
    template_path=template_path
)
all_levels

C:\Users\loftuw\Documents\Github\calibrated_explanations\src\calibrated_explanations\explanations\explanations.py:1098: UserWarning: Narrative template fallback: default template used because provided relative path was missing
  return plugin.plot(


,instance_index,factual_explanation_beginner,factual_explanation_intermediate,factual_explanation_advanced,expertise_level,problem_type
0,0,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, intermediate, advanced)",binary_classification
1,1,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, intermediate, advanced)",binary_classification
2,2,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, intermediate, advanced)",binary_classification
3,3,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, intermediate, advanced)",binary_classification
4,4,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, intermediate, advanced)",binary_classification
5,5,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, intermediate, advanced)",binary_classification
6,6,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, intermediate, advanced)",binary_classification
7,7,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, intermediate, advanced)",binary_classification
8,8,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, intermediate, advanced)",binary_classification
9,9,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, intermediate, advanced)",binary_classification


In [13]:
# Template path handling:

# If exp.yaml doesn't exist, automatically falls back to explain_template.yaml
narratives = explanations.to_narrative(
    template_path="exp.yaml",  # Will use default if not found
    expertise_level=("beginner", "advanced"),
    output_format="dataframe"
)
narratives

C:\Users\loftuw\Documents\Github\calibrated_explanations\src\calibrated_explanations\explanations\explanations.py:1098: UserWarning: Narrative template fallback: default template used because provided relative path was missing
  return plugin.plot(


,instance_index,factual_explanation_beginner,factual_explanation_advanced,expertise_level,problem_type
0,0,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
1,1,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
2,2,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
3,3,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
4,4,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
5,5,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
6,6,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
7,7,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
8,8,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification
9,9,Prediction: Diabetic\nCalibrated Probability: ...,Prediction: Diabetic\nCalibrated Probability: ...,"(beginner, advanced)",binary_classification


In [14]:
# Use custom template
narratives = explanations.to_narrative(
    template_path="/path/to/custom_template.yaml",
    expertise_level=("beginner", "advanced"),
    output_format="dataframe"
)

# Use default template explicitly
narratives = explanations.to_narrative(
    template_path=template_path,
    expertise_level=("beginner", "advanced"),
    output_format="dataframe"
)

print("The to_narrative method provides a clean, intuitive API for generating narratives!")

The to_narrative method provides a clean, intuitive API for generating narratives!


C:\Users\loftuw\Documents\Github\calibrated_explanations\src\calibrated_explanations\explanations\explanations.py:1098: UserWarning: Narrative template fallback: default template used because provided relative path was missing
  return plugin.plot(
C:\Users\loftuw\Documents\Github\calibrated_explanations\src\calibrated_explanations\explanations\explanations.py:1098: UserWarning: Narrative template fallback: default template used because provided relative path was missing
  return plugin.plot(
